<a href="https://colab.research.google.com/github/jonschumaker/DSE6000_plotly/blob/main/dash_charts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#above line is not needed for colab but needed for non colab environments to show the plot 
sns.set(color_codes=True)
!pip install --upgrade plotly
import plotly.express as px

     |████████████████████████████████| 25.3 MB 1.6 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
#df = pd.read_csv('https://drive.google.com/uc?export=download&id=1oB1qEc360zOocTPAnxcrRSqzyy__E5zE')
#df.columns = df.columns.str.strip()

In [3]:
#df1 = pd.read_csv('simplyhired_cleaned_G_K_A_J.csv')
df = pd.read_csv('https://drive.google.com/uc?export=download&id=1fs6PHcGNLhB0jVDGMvKanQxevXQh6czC')
df.columns = df.columns.str.strip()

This code should be run prior to files being combined.

Run next blocks of code for Indeed file to clean the location column.

In [4]:
#Indeed dataset: Changes 'location' column to 3 columns: 'remote', 'city', and 'state'

#df['remote'] = np.where(df['location'].str.contains(r'Remote'), 'yes', 'no')
#df['location'] = df['location'].map(lambda x: x.lstrip('+-').rstrip('Remote'))
#df['location'] = df['location'].replace({'•': ''}, regex=True)
#df['city'], df['trash'] = df['location'].str.rsplit(', ', 1).str
#df['state'] = df['trash'].str[:2]
#df['state'] = df['state'].replace("A", "AR")
#df['state'] = df['state'].replace("O", "OR")
#df.head()

In [5]:
#df.drop(['trash', 'jobtype', 'location'], axis = 1, inplace = True)

Code blocks for Simply Hired file to clean the location column.

In [6]:
#df['remote'] = np.where(df['location'].str.contains(r'Remote'), 'yes', 'no')

In [7]:
#df['city'], df['trash'] = df['location'].str.rsplit(', ', 1).str
#df['state'] = df['trash'].str[:2]
#df.drop(['trash', 'location'], axis = 1, inplace = True)
#df['city'] = df['city'].str.strip()

In [8]:
#Run for both Indeed and Simply after combined or individually changing df to df1
#def Clean_city(City_name):
    # Search for opening bracket in the name followed by
    # any characters repeated any number of times
#    if re.search('\+.*', City_name):
  
        # Extract the position of beginning of pattern
#        pos = re.search('\+.*', City_name).start()
  
        # return the cleaned name
#        return City_name[:pos]
  
#    else:
        # if clean up needed return the same name
#        return City_name
          
# Updated the city columns
#df['city'] = df['city'].apply(Clean_city)

Code block to clean salary column.

In [9]:
class salary:
    def __init__(self, base, max, time):
        self.base = base
        self.max = max
        self.time = time

    def pay(self):
        return pay(self)
    

def pay(sal):
    if sal.base == False:
        sal.pay = sal.max * sal.time
    elif sal.max == True:
        sal.pay = sal.base * sal.time
    else:
        sal.pay = (sal.base + sal.max) / 2 * sal.time
    return sal.pay

def parse_salary(arr):
    arr = arr.replace('$', '')
    arr = arr.replace(',', '').lower()
    arr = arr.split()
    base_salary = False
    max_salary = False
    for i in arr:
        try:
            i = float(i)
            if max_salary:
                max_salary = i
            else:
                base_salary = i
                max_salary = True
        except:
            if i == 'up':
                max_salary = True
        
        if i == 'hour':
            rate = 2080
        elif i == 'month':
            rate = 12
        elif i == 'year':
            rate = 1

    return salary(base = base_salary, max = max_salary, time = rate)

def parse_pay(arr):
    new_sal = []
    for i in arr:
        try:
            new_sal.append(pay(parse_salary(i)))
        except:
            new_sal.append(float("NaN"))
    return new_sal

def parse_file(csv):
    df = pd.read_csv(csv)
    salary_list = df['salary']
    salary_list = list(salary_list)
    return parse_pay(salary_list)

df['salary'] = parse_pay(df['salary'])

In [10]:
df.shape

(5558, 10)

In [11]:
df = df.drop_duplicates()
df.shape

(1875, 10)

In [12]:
df

,title,salary,description,remote,city,state,job_board,company,compan,industry
0,Data Scientist,NaN,Experience with eCommerce data analysis is a p...,yes,Delano,CA,Indeed,NaN,NaN,NaN
1,Data Scientist,NaN,NaN,yes,Remote,NaN,Indeed,NaN,NaN,NaN
2,Data Scientist,NaN,Experience supporting data scientists across t...,yes,Remote,NaN,Indeed,NaN,NaN,NaN
3,Data Engineer,NaN,Define and execute the data engineering roadma...,no,Atlanta,GA,Indeed,NaN,NaN,NaN
4,Data Scientist,NaN,"Understands how to break down the data, extrac...",yes,Seattle,WA,Indeed,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5553,Data Scientist,NaN,Train and tune a variety of model architecture...,yes,Remote,NaN,Simply Hired,XSOLIS,XSOLIS,NaN
5554,Data Scientist,NaN,Partner with data architects to ensure that Da...,no,New York,NY,Simply Hired,Yelp,Yelp,Entertainment
5555,Data Analyst,NaN,"An undergraduate degree in computer science, m...",no,New York,NY,Simply Hired,YieldX,YieldX,NaN
5556,Data Scientist,NaN,1-3 years of experience in data science and ad...,yes,Remote,NaN,Simply Hired,Zebit,Zebit,NaN


In [13]:
ind_df = df.groupby(['industry'])
ind_sal = ind_df.mean()
ind = ind_df.count()
ind.rename(columns={'salary':'salary_ind'}, inplace=True)
ind['ratio'] = ind['salary_ind'] / ind['title']
ind = pd.concat([ind, ind_sal], axis = 1).reset_index()
fig = px.treemap(ind, path = ['industry'], values = 'title', color = 'salary', color_continuous_scale='Emrld')
fig.show()

In [14]:
fig = px.treemap(ind, path = ['industry'], values = 'title', color = 'ratio', color_continuous_scale='Burg')
fig.show()

In [15]:
nonnan = df[df['industry'] == df['industry']]
nonnan

,title,salary,description,remote,city,state,job_board,company,compan,industry
2274,Data Engineer,NaN,"Proficiency in Data Engineering frameworks, ab...",no,Irving,TX,Simply Hired,"7-Eleven, Inc.",7-Elev,Retail (Grocery and Food)
2276,Data Analyst,97000.0,Promote data literacy and a data driven decisi...,no,Nashville,TN,Simply Hired,A. O. Smith Corporation,A. O.,Building Materials
2279,Data Scientist,NaN,"Expertise with developing, implementing and de...",no,Lake County,IL,Simply Hired,Abbvie,Abbvie,Drugs (Biotechnology)
2280,Data Scientist,NaN,Primary objectives are to monetize our custome...,no,Columbus,OH,Simply Hired,Abercrombie and Fitch Co.,Abercr,Retail (Special Lines)
2491,Data Scientist,NaN,Foster the adoption of NLP and data science ac...,no,Atlanta,GA,Simply Hired,Accenture,Accent,Retail (Special Lines)
...,...,...,...,...,...,...,...,...,...,...
5539,Data Scientist,170500.0,"Expert experience with data pipelines, data to...",no,Bethesda,MD,Simply Hired,Wellstone Technologies,Wellst,Healthcare Products
5546,Data Analyst,NaN,We help them make data analysis a key part to ...,yes,Remote,NaN,Simply Hired,"World Wide Technology, Inc.",World,Financial Svcs. (Non-bank & Insurance)
5547,Data Analyst,NaN,We take on the most challenging problems our c...,yes,Remote,NaN,Simply Hired,"World Wide Technology, Inc.",World,Financial Svcs. (Non-bank & Insurance)
5554,Data Scientist,NaN,Partner with data architects to ensure that Da...,no,New York,NY,Simply Hired,Yelp,Yelp,Entertainment


In [16]:
px.treemap(nonnan, path = ['title','industry'], color_continuous_scale='Emrld').show()

In [17]:
size_df = df.groupby(['title', 'remote']).size()
sal_df = df.groupby(['title', 'remote']).mean()
gdf = pd.concat([size_df, sal_df], axis = 1).reset_index()
fig = px.treemap(gdf, path = ['title', 'remote'], values = 0, color='salary', color_continuous_scale='Emrld')
fig.show()

In [18]:
df.to_csv('combined_G_K_A_J.csv', encoding = 'utf-8-sig', index = False)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1875 entries, 0 to 5557
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        1875 non-null   object 
 1   salary       344 non-null    float64
 2   description  1869 non-null   object 
 3   remote       1875 non-null   object 
 4   city         1871 non-null   object 
 5   state        1501 non-null   object 
 6   job_board    1875 non-null   object 
 7   company      998 non-null    object 
 8   compan       998 non-null    object 
 9   industry     541 non-null    object 
dtypes: float64(1), object(9)
memory usage: 161.1+ KB


In [20]:
jobs_count_state = df.groupby(['state'])
jobs_count_state = jobs_count_state.size()
jobs_count_state = jobs_count_state.to_frame()
jobs_count_state.reset_index(inplace=True)
jobs_count_state.columns = ['state', 'count']
print(jobs_count_state.head())

  state  count
0    AK      2
1    AL      9
2    AR      6
3    AZ      7
4    CA    241


In [21]:
jobs_count_state.to_csv('jobs_count_state.csv', encoding = 'utf-8-sig', index = False)

In [22]:
groups = df.groupby(['state', 'title'])
groups = groups.size()
groups.head()

state  title         
AK     Statistician      2
AL     Data Analyst      2
       Data Engineer     1
       Data Scientist    4
       Data, Other       1
dtype: int64

In [23]:
groups = groups.to_frame()
groups.head(5)

0
state title            
AK    Statistician    2
AL    Data Analyst    2
      Data Engineer   1
      Data Scientist  4
      Data, Other     1

In [24]:
groups.reset_index(inplace=True)
groups.columns = ['state', 'title', 'title_count']
print(groups.head())

  state           title  title_count
0    AK    Statistician            2
1    AL    Data Analyst            2
2    AL   Data Engineer            1
3    AL  Data Scientist            4
4    AL     Data, Other            1


In [25]:
groups.to_csv('groups.csv', encoding = 'utf-8-sig', index = False)

In [26]:
fig = px.choropleth(groups, locations='state', locationmode="USA-states", color='title_count',
                             scope="usa", color_continuous_scale = 'tempo',
                             labels={'count':'Job Listings'})
fig.show()

In [27]:
agg_5 = df['title'].value_counts()[:5]
fig_bar = px.bar(agg_5, title = "Top 5 Job Titles",
                labels=None)
plt.tight_layout()
fig_bar.show()
df.head()

,title,salary,description,remote,city,state,job_board,company,compan,industry
0,Data Scientist,NaN,Experience with eCommerce data analysis is a p...,yes,Delano,CA,Indeed,NaN,NaN,NaN
1,Data Scientist,NaN,NaN,yes,Remote,NaN,Indeed,NaN,NaN,NaN
2,Data Scientist,NaN,Experience supporting data scientists across t...,yes,Remote,NaN,Indeed,NaN,NaN,NaN
3,Data Engineer,NaN,Define and execute the data engineering roadma...,no,Atlanta,GA,Indeed,NaN,NaN,NaN
4,Data Scientist,NaN,"Understands how to break down the data, extrac...",yes,Seattle,WA,Indeed,NaN,NaN,NaN


<Figure size 432x288 with 0 Axes>

In [28]:
#TITLE AND SALARY
title_salary = df[['title', 'salary']]
title_salary = title_salary.dropna()
title_salary.head(10)

,title,salary
13,Data Scientist,72800.0
40,Data Scientist,92250.0
42,Data Analyst,31200.0
45,Statistician,72800.0
52,Data Scientist,68162.0
75,Data Scientist,112078.5
86,Data Scientist,114660.0
90,Data Scientist,191000.0
92,Statistician,45638.0
118,Data Analyst,77862.0


In [29]:
title_salary = title_salary.dropna()
title_salary.head(20)

,title,salary
13,Data Scientist,72800.0
40,Data Scientist,92250.0
42,Data Analyst,31200.0
45,Statistician,72800.0
52,Data Scientist,68162.0
75,Data Scientist,112078.5
86,Data Scientist,114660.0
90,Data Scientist,191000.0
92,Statistician,45638.0
118,Data Analyst,77862.0


In [30]:
title_salary.to_csv('title_salary.csv', encoding = 'utf-8-sig', index = False)

Jon's Crap below:

In [31]:
df['title']= df['title'].str.lower()
df.groupby('title').size().sort_values(ascending=False)
df_count = pd.DataFrame(df.groupby('title').size().sort_values(ascending=False).rename('Count').reset_index())
df_count

,title,Count
0,data scientist,881
1,data analyst,361
2,data engineer,228
3,"data, other",89
4,statistician,73
5,machine learning,66
6,analytics,43
7,education,33
8,manager,26
9,ai,26


In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
wordcloud = WordCloud(width = 5000, height = 3000, colormap="Reds").generate(" ".join(df.title))
plt.imshow(wordcloud,interpolation="bilinear")
plt.title('Cool Word Cloud of Job Titles', fontsize=15, color ="Black", fontweight="bold")
plt.axis('off')
plt.show()


In [ ]:
#Industry and Salary
industry_salary_df = df[['industry', 'salary']]
industry_salary_df = industry_salary_df.dropna()
industry_salary_df.head(10)

Hypothesis: We expect to see words related to data in the most frequently used words for the job title 'Data Scientist'
Result: As expected, all five of the most frequently used words for 'Data Scientist' contain the word data.

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
stop_words = ['NaN','NaN ','in','of','you','year','Simply Hired','Simply','Indeed','Data Scientist','Scientist','Hired','Seattle','Columbus','WA','OH','Portland','CA']
wordcloud = WordCloud(max_words = 20, stopwords = stop_words, width = 5000, height = 3000, colormap="Reds").generate(df.loc[df['title']=='Data Scientist'].to_string())
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Data Scientist Word Cloud', fontsize=15, color ="Black", fontweight="bold")
plt.axis("off")
plt.show()
plt.savefig('data_scientist_word_cloud.png')

# New Section

In [ ]:
sorted(wordcloud.words_.items(), key=lambda x: x[1], reverse=True)[:5]


In [ ]:
stop_words = ['NaN','NaN ','Simply Hired','Simply','Indeed','Data Analyst','Analyst','Hired','Alpharetta']
wordcloud = WordCloud(max_words = 20, stopwords = stop_words, width = 5000, height = 3000, colormap="Reds").\
generate(df.loc[df['title']=='Data Analyst'].to_string())
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Data Analyst Word Cloud', fontsize=15, color ="Black", fontweight="bold")
plt.axis("off")
plt.show()

In [ ]:
sorted(wordcloud.words_.items(), key=lambda x: x[1], reverse=True)[:5]

In [ ]:
stop_words = ['NaN','NaN ','and','no','the','Simply Hired','Simply','Indeed','Data Engineer','Engineer','Hired','New York','NY','York','Ampersand','Ampers']
wordcloud = WordCloud(max_words = 20, stopwords = stop_words, width = 5000, height = 3000, colormap="Reds").\
generate(df.loc[df['title']=='Data Engineer'].to_string())
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Data Engineer Word Cloud', fontsize=15, color ="Black", fontweight="bold")
plt.axis("off")
plt.show()

In [ ]:
sorted(wordcloud.words_.items(), key=lambda x: x[1], reverse=True)[:5]

In [ ]:
stop_words = ['NaN','NaN ','Simply Hired','Simply','Indeed','Other']
wordcloud = WordCloud(max_words = 20, stopwords = stop_words, width = 5000, height = 3000, colormap="Reds").\
generate(df.loc[df['title']=='Data, Other'].to_string())
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Data, Other Word Cloud', fontsize=15, color ="Black", fontweight="bold")
plt.axis("off")
plt.show()

In [ ]:
stop_words = ['NaN','NaN ','Simply Hired','Simply','Indeed','Hired','PA','Pfizer','Statistician']
wordcloud = WordCloud(max_words = 20, stopwords = stop_words, width = 5000, height = 3000, colormap="Reds").\
generate(df.loc[df['title']=='Statistician'].to_string())
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Statistician Word Cloud', fontsize=15, color ="Black", fontweight="bold")
plt.axis("off")
plt.show()